In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Bat ML

In [2]:
bat_df = pd.read_csv("../datasets/bat.csv")
match_df = pd.read_csv("../datasets/mat.csv", parse_dates=["match_date"])
opposition_df = pd.read_csv("../datasets/opposition.csv")
player_df = pd.read_csv("../datasets/player.csv")
ground_df = pd.read_csv("../datasets/ground.csv")

In [3]:
bat_df.shape

(3934, 8)

In [4]:
bat_df.describe()

,player,mat,runs,ball,M,_4s,_6s,strike_rate
count,3934.000000,3.934000e+03,3934.000000,3934.000000,3934.000000,3934.000000,3934.000000,3934.000000
mean,95981.080834,4.247568e+05,27.982715,33.500508,42.209202,2.531774,0.375191,77.576002
std,126650.660419,3.550217e+05,30.470857,32.189661,50.800320,3.502842,1.884419,49.118890
min,40618.000000,6.465700e+04,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000
25%,44932.000000,6.578700e+04,5.000000,9.000000,11.000000,0.000000,0.000000,50.000000
50%,45789.000000,3.503480e+05,17.000000,23.000000,29.000000,1.000000,0.000000,75.000000
75%,47015.000000,6.490990e+05,41.000000,47.750000,64.000000,4.000000,0.000000,100.000000
max,698189.000000,1.185321e+06,185.000000,160.000000,257.000000,21.000000,16.000000,460.000000


In [5]:
# Prepare for merging
bat_df.rename(columns={"player":"player_id", "mat":"match_id"}, inplace=True)
match_df.rename(columns={"opposition":"opposition_id", "ground":"venue_id"}, inplace=True)
opposition_df.rename(columns={"opp_id":"opposition_id", "opp_name":"opposition_name", "rating":"opposition_rating"}, inplace=True)
ground_df.rename(columns={"ground_id":"venue_id", "ground_name":"venue_name", "country":"venue_country"}, inplace=True)
player_df.rename(columns={"name":"player_name"}, inplace=True)

In [6]:
df = bat_df.merge(
    player_df, how="left", on="player_id"
).merge(
    match_df, how="left", on="match_id"
).merge(
    ground_df, how="left", on="venue_id"
).merge(
    opposition_df, how="left", on="opposition_id"
)

print(f"Data shape: {df.shape}")

Data shape: (3934, 26)


In [7]:
df.columns

Index(['player_id', 'match_id', 'runs', 'ball', 'M', '_4s', '_6s',
       'strike_rate', 'player_name', 'odi_debut', 'playing_role',
       'batting_style', 'bowling_style', 'fielding_position', 'odi_no',
       'opposition_id', 'venue_id', 'match_date', 'toss', 'series', 'result',
       'match_days', 'venue_name', 'venue_country', 'opposition_name',
       'opposition_rating'],
      dtype='object')

### Data cleaning
* remove duplicates (note: np.nan != np.nan)
* Remove absent hurt (balls==0)
* sort by date (reindex too)
* nan is -99 for most columns, replace with 0

In [8]:
print(f"Old shape: {df.shape}")

# Drop duplicates
df.drop_duplicates(["player_id", "match_id", "runs", "ball"], inplace=True)

# Remove absent hurt
df.drop(df[df["ball"]==0].index, inplace=True)

print(f"New shape: {df.shape}")

Old shape: (3934, 26)
New shape: (3449, 26)


In [9]:
# Sort by match date
df.sort_values("match_date", axis=0, ignore_index=True, inplace=True)

# Replace -99 with 0
df.replace(-99, 0, inplace=True)

In [10]:
df.describe()

,player_id,match_id,runs,ball,M,_4s,_6s,strike_rate,odi_no,opposition_id,venue_id,opposition_rating
count,3449.000000,3.449000e+03,3449.000000,3449.000000,3449.000000,3449.000000,3449.000000,3449.000000,3449.000000,3449.000000,3.449000e+03,3449.000000
mean,98855.237170,4.297030e+05,28.188460,33.603653,44.907219,2.582198,0.403885,77.761305,2771.299507,6.553784,7.869456e+04,96.781096
std,130750.395555,3.653009e+05,30.525281,32.191617,45.710885,3.151572,1.022427,48.913158,819.934830,5.762867,8.430320e+04,26.356283
min,40618.000000,6.465700e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1544.000000,1.000000,5.629300e+04,0.000000
25%,44932.000000,6.609700e+04,5.000000,9.000000,10.000000,0.000000,0.000000,50.000000,2030.000000,2.000000,5.879200e+04,85.000000
50%,45789.000000,2.990090e+05,17.000000,23.000000,29.000000,1.000000,0.000000,75.000000,2691.000000,6.000000,5.908900e+04,102.000000
75%,47255.000000,7.223390e+05,41.000000,48.000000,65.000000,4.000000,0.000000,100.000000,3522.000000,8.000000,5.930600e+04,116.000000
max,698189.000000,1.185321e+06,185.000000,160.000000,257.000000,21.000000,16.000000,460.000000,4254.000000,40.000000,1.126475e+06,125.000000


Null values in Opposition_rating has to be replaced with their correct value. I couldn't find ODI men rating for Canada and Kenya, so we'll keep them at 0. [1](https://www.icc-cricket.com/rankings/mens/team-rankings/odi) and [2](https://sports.ndtv.com/cricket/icc-rankings) gives Netherlands a rating of 44

In [11]:
opposition_df[opposition_df["opposition_rating"]==-99]

,opposition_id,opposition_name,opposition_rating
8,15,Netherlands,-99
9,17,Canada,-99
11,26,Kenya,-99


In [12]:
# Replace Netherlands rating to 44
df.loc[df["opposition_name"]=="Netherlands", 'opposition_rating'] = 44

Lastly, probably worth noting that strike rate was **rounded down** for runs/ball.

### Batting attributes
* No. of innings = count(player appearance before that match)
* Batting average = sum(runs) / No. of innings
* Strike rate = 100 * sum(runs) / sum(ball)
* fifties = count(innings where 50 <= run < 100 )
* centuries = count(innings where 100 <= run)
* zeros = count(innings where run==0)
* highest score = max(runs)


### Derived attributes 
* Consistency: entire career
* Form: last 12 months
* Opposition: opposition team
* Venue: venue

In [13]:
# The training start from 2017 because all odi_debut for players after 2016 is more than the start date(2000) of df
train = df.loc[df["match_date"] >= "2017"]

print(f"Train shape: {train.shape}")

Train shape: (458, 26)


In [14]:
train.head()

,player_id,match_id,runs,ball,M,_4s,_6s,strike_rate,player_name,odi_debut,...,venue_id,match_date,toss,series,result,match_days,venue_name,venue_country,opposition_name,opposition_rating
2991,379143,936159,34,40,56,4,0,85.00,Quinton de Kock,2013-01-19,...,59159,2017-01-28,"South Africa , elected to field first",Sri Lanka tour of South Africa,South Africa led the 5-match series 1-0,28 January 2017 (50-over match),St George's Park,South Africa,Sri Lanka,85
2992,43906,936159,57,71,110,5,1,80.28,Hashim Amla,2008-03-09,...,59159,2017-01-28,"South Africa , elected to field first",Sri Lanka tour of South Africa,South Africa led the 5-match series 1-0,28 January 2017 (50-over match),St George's Park,South Africa,Sri Lanka,85
2993,44828,936159,55,68,85,5,0,80.88,Faf du Plessis,2011-01-18,...,59159,2017-01-28,"South Africa , elected to field first",Sri Lanka tour of South Africa,South Africa led the 5-match series 1-0,28 January 2017 (50-over match),St George's Park,South Africa,Sri Lanka,85
2994,44936,936159,30,27,32,3,1,111.11,AB de Villiers,2005-02-02,...,59159,2017-01-28,"South Africa , elected to field first",Sri Lanka tour of South Africa,South Africa led the 5-match series 1-0,28 January 2017 (50-over match),St George's Park,South Africa,Sri Lanka,85
2995,44932,936161,11,19,27,0,0,57.89,Jean-Paul Duminy,2004-08-20,...,59089,2017-02-01,"Sri Lanka , elected to field first",Sri Lanka tour of South Africa,South Africa led the 5-match series 2-0,1 February 2017 - day/night match (50-over match),Kingsmead,South Africa,Sri Lanka,85


In [15]:
def get_basic_attributes(df, bins, labels):
    no_of_innings = df.shape[0]
    if no_of_innings==0:
        return 0,0,0,0,0,0,0
    
    runs_scored = df["runs"].sum()
    balls_faced = df["ball"].sum()
    batting_average = np.around(runs_scored / no_of_innings, 2)
    strike_rate = np.around((runs_scored/balls_faced) * 100, 2)
    centuries = df[df["runs"] >= 100].shape[0]
    fifties = df[(df["runs"]>=50) & (df["runs"]<100)].shape[0]
    zeros = df[df["runs"]==0].shape[0]
    highest_score = df["runs"].max()
    
    no_of_innings = pd.cut([no_of_innings], bins["no_of_innings"], right=False, labels=labels["no_of_innings"])[0]
    batting_average = pd.cut([batting_average], bins["batting_average"], right=False, labels=labels["batting_average"])[0]
    strike_rate = pd.cut([strike_rate], bins["strike_rate"], right=False, labels=labels["strike_rate"])[0]
    centuries = pd.cut([centuries], bins["centuries"], right=False, labels=labels["centuries"])[0]
    fifties = pd.cut([fifties], bins["fifties"], right=False, labels=labels["fifties"])[0]
    if bins["zeros"]:
        zeros = pd.cut([zeros], bins["zeros"], right=False, labels=labels["zeros"])[0]
    if bins["highest_score"]:
        highest_score = pd.cut([highest_score], bins["highest_score"], right=False, labels=labels["highest_score"])[0]
    
    
    return no_of_innings, batting_average, strike_rate, centuries, fifties, zeros, highest_score

In [16]:
def consistency_attribute(row):
    mask = (
        (df["player_id"] == row["player_id"])
        & (df["match_date"] < row["match_date"])
    )
    consistency_df = df[mask]
    
    bins = {
        "no_of_innings": [1,50,100,125,150,np.inf],
        "batting_average": [0,10,20,30,40,np.inf],
        "strike_rate": [0,50,60,80,100,np.inf],
        "centuries": [0,1,5,10,15,20,np.inf],
        "fifties": [0,1,10,20,30,40,np.inf],
        "zeros": [0,1,5,10,15,20,np.inf],
        "highest_score": None,
    }
    labels = {
        "no_of_innings": [1,2,3,4,5],
        "batting_average": [1,2,3,4,5],
        "strike_rate": [1,2,3,4,5],
        "centuries": [0,1,2,3,4,5],
        "fifties": [0,1,2,3,4,5],
        "zeros": [0,1,2,3,4,5],
        "highest_score": None,
    }
    
    no_of_innings, batting_average, strike_rate, centuries, fifties, zeros, highest_score = get_basic_attributes(consistency_df, bins, labels)
    consistency = (0.4262*batting_average + 0.2566*no_of_innings + 0.1510*strike_rate + 0.0787*centuries + 0.0556*fifties - 0.0328*zeros)
    
    return np.around(consistency, 4)

In [17]:
def form_attribute(row):
    mask = (
        (df["player_id"] == row["player_id"])
        & (df["match_date"] < row["match_date"])
        & (row["match_date"]-df["match_date"] <= np.timedelta64(1, "Y"))
    )
    form_df = df[mask]
    bins = {
        "no_of_innings": [1,5,10,12,15,np.inf],
        "batting_average": [0,10,20,30,40,np.inf],
        "strike_rate": [0,50,60,80,100,np.inf],
        "centuries": [0,1,2,3,4,5,np.inf],
        "fifties": [0,1,3,5,7,10,np.inf],
        "zeros": [0,1,2,3,4,5,np.inf],
        "highest_score": None,
    }
    labels = {
        "no_of_innings": [1,2,3,4,5],
        "batting_average": [1,2,3,4,5],
        "strike_rate": [1,2,3,4,5],
        "centuries": [0,1,2,3,4,5],
        "fifties": [0,1,2,3,4,5],
        "zeros": [0,1,2,3,4,5],
        "highest_score": None,
    }
    
    no_of_innings, batting_average, strike_rate, centuries, fifties, zeros, highest_score = get_basic_attributes(form_df, bins, labels)
    form = (0.4262*batting_average + 0.2566*no_of_innings + 0.1510*strike_rate + 0.0787*centuries + 0.0556*fifties - 0.0328*zeros)
    
    return np.around(form, 4)

In [18]:
def opposition_attribute(row):
    mask = (
        (df["player_id"] == row["player_id"])
        & (df["match_date"] < row["match_date"])
        & (df["opposition_id"] == row["opposition_id"])
    )
    opposition_df = df[mask]
    
    bins = {
        "no_of_innings": [1,3,5,7,10,np.inf],
        "batting_average": [0,10,20,30,40,np.inf],
        "strike_rate": [0,50,60,80,100,np.inf],
        "centuries": [0,1,2,3,np.inf],
        "fifties": [0,1,3,5,7,10,np.inf],
        "zeros": [0,1,2,3,4,5,np.inf],
        "highest_score":  None,
    }
    labels = {
        "no_of_innings": [1,2,3,4,5],
        "batting_average": [1,2,3,4,5],
        "strike_rate": [1,2,3,4,5],
        "centuries": [0,3,4,5],
        "fifties": [0,1,2,3,4,5],
        "zeros": [0,1,2,3,4,5],
        "highest_score": None,
    }
    
    no_of_innings, batting_average, strike_rate, centuries, fifties, zeros, highest_score = get_basic_attributes(opposition_df, bins, labels)
    opposition = (0.4262*batting_average + 0.2566*no_of_innings + 0.1510*strike_rate + 0.0787*centuries + 0.0556*fifties - 0.0328*zeros)
    
    return np.around(opposition, 4)

In [19]:
def venue_attribute(row):
    mask = (
        (df["player_id"] == row["player_id"])
        & (df["match_date"] < row["match_date"])
        & (df["venue_country"] == row["venue_country"])
    )
    venue_df = df[mask]
    
    bins = {
        "no_of_innings": [1,2,3,4,5,np.inf],
        "batting_average": [0,10,20,30,40,np.inf],
        "strike_rate": [0,50,60,80,100,np.inf],
        "centuries": [0,1,2,np.inf],
        "fifties": [0,1,2,np.inf],
        "zeros": None,
        "highest_score": [0,1,25,50,100,150,np.inf],
    }
    labels = {
        "no_of_innings": [1,2,3,4,5],
        "batting_average": [1,2,3,4,5],
        "strike_rate": [1,2,3,4,5],
        "centuries": [0,4,5],
        "fifties": [0,4,5],
        "zeros": None,
        "highest_score": [0,1,2,3,4,5],
    }
    
    no_of_innings, batting_average, strike_rate, centuries, fifties, zeros, highest_score = get_basic_attributes(venue_df, bins, labels)
    venue = (0.4262*batting_average + 0.2566*no_of_innings + 0.1510*strike_rate + 0.0787*centuries + 0.0556*fifties + 0.0328*highest_score)
    
    return np.around(venue,4)

In [20]:
X = pd.DataFrame()
X["consistency"] = train.apply(consistency_attribute, axis=1)
X["form"] = train.apply(form_attribute, axis=1)
X["opposition"] = train.apply(opposition_attribute, axis=1)
X["venue"] = train.apply(venue_attribute, axis=1)
print(f"X shape: {X.shape}")

X shape: (458, 4)


In [21]:
# Personally, I think strike_rate is a better target for measuring performance.
# The models perform poorly for strike_rate though
y = train.loc[:, "runs"]

#### Data cleaning

Change 0s to mean

In [22]:
X.loc[X["consistency"]==0, "consistency"] = X[X["consistency"]!=0]["consistency"].mean()
X.loc[X["form"]==0, "form"] = X[X["form"]!=0]["form"].mean()
X.loc[X["opposition"]==0, "opposition"] = X[X["opposition"]!=0]["opposition"].mean()
X.loc[X["venue"]==0, "venue"] = X[X["venue"]!=0]["venue"].mean()

In [23]:
X.describe()

,consistency,form,opposition,venue
count,458.000000,458.000000,458.000000,458.000000
mean,2.785411,2.931287,2.990929,3.507608
std,1.083639,0.997357,1.061367,1.089377
min,0.801000,0.801000,0.801000,0.833800
25%,1.895000,2.249000,2.193400,2.805000
50%,2.785411,2.931543,2.990929,3.507608
75%,3.562700,3.639100,3.929400,4.394500
max,4.774900,4.548800,4.784900,5.004500


### Algorithm: Regression models
The paper used classification and measured accuracy.

In [24]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

In [25]:
model = LinearRegression()
score = cross_val_score(model, X.values, y.values, scoring="neg_mean_squared_error", n_jobs=-1)
print(f"Score: {score}\nMean: {score.mean()}")

Score: [-1063.33486834  -985.29811344  -525.32598422 -1002.76693727
  -925.15152331]
Mean: -900.3754853155733


In [26]:
model = SVR()
score = cross_val_score(model, X.values, y.values, scoring="neg_mean_squared_error", n_jobs=-1)
print(f"Score: {score}\nMean: {score.mean()}")

Score: [-1198.19676241 -1008.01238709  -504.96110279 -1046.83525919
 -1064.1771652 ]
Mean: -964.4365353338126


In [27]:
model = DecisionTreeRegressor()
score = cross_val_score(model, X.values, y.values, scoring="neg_mean_squared_error", n_jobs=-1)
print(f"Score: {score}\nMean: {score.mean()}")

Score: [-2217.8964372  -2055.72484729  -987.45048309 -1654.73197777
 -1893.31046956]
Mean: -1761.8228429822836


In [28]:
model = RandomForestRegressor()
score = cross_val_score(model, X.values, y.values, scoring="neg_mean_squared_error", n_jobs=-1)
print(f"Score: {score}\nMean: {score.mean()}")

Score: [-1248.14841272 -1057.97715538  -727.42180527 -1108.97082252
  -993.98473101]
Mean: -1027.3005853821473


In [29]:
model = GradientBoostingRegressor()
score = cross_val_score(model, X.values, y.values, scoring="neg_mean_squared_error", n_jobs=-1)
print(f"Score: {score}\nMean: {score.mean()}")

Score: [-1209.5147564  -1144.27807393  -810.53949694 -1081.08174531
 -1147.22325291]
Mean: -1078.527465097342


#### There are other input attributes but it wasn't included because the basic attribute is decent enough and our dataset is small. For illustration, here it is.

In [30]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [31]:
df.columns

Index(['player_id', 'match_id', 'runs', 'ball', 'M', '_4s', '_6s',
       'strike_rate', 'player_name', 'odi_debut', 'playing_role',
       'batting_style', 'bowling_style', 'fielding_position', 'odi_no',
       'opposition_id', 'venue_id', 'match_date', 'toss', 'series', 'result',
       'match_days', 'venue_name', 'venue_country', 'opposition_name',
       'opposition_rating'],
      dtype='object')

In [32]:
# Use sample or unique method to understand column values
df["toss"].sample(10)

3052      South Africa , elected to bat first
174     South Africa , elected to field first
3269    South Africa , elected to field first
196        Sri Lanka , elected to field first
2773      South Africa , elected to bat first
1976      South Africa , elected to bat first
2696     West Indies , elected to field first
23        South Africa , elected to bat first
2779      South Africa , elected to bat first
1310         Australia , elected to bat first
Name: toss, dtype: object

In [33]:
def batting_hand(bat_style):
    # unique() = ['Left-hand bat', 'Right-hand bat']
    if "Right-hand" in bat_style:
        return 1
    elif "Left-hand" in bat_style:
        return 0
    else:
        return -1
    
X["batting_hand"] = train["batting_style"].apply(batting_hand)
X["batting_hand"].value_counts()

1    276
0    182
Name: batting_hand, dtype: int64

In [34]:
def bowling_hand(bowl_style):
    # There are nan in bowl_style
    if type(bowl_style)==float:
        return -2
    elif "right-arm" in bowl_style.lower():
        return 1
    elif "left-arm" in bowl_style.lower():
        return 0
    else:
        return -1
    
X["bowling_hand"] = train["bowling_style"].apply(bowling_hand)
X["bowling_hand"].value_counts()

 1    297
-1     73
-2     66
 0     22
Name: bowling_hand, dtype: int64

In [35]:
def match_time(arg):
    # There are nan in bowl_style
    if "day/night" in arg:
        return 1
    else:
        return 0
    
X["match_time"] = train["match_days"].apply(match_time)
X["match_time"].value_counts()

1    266
0    192
Name: match_time, dtype: int64

In [36]:
def venue(country):
    if country=="South Africa":
        return 1
    else:
        return 0

X["home_venue"] = train["venue_country"].apply(venue)
X["home_venue"].value_counts()

1    234
0    224
Name: home_venue, dtype: int64

In [37]:
def toss_attribute(toss):
    if "South Africa" in toss:
        return 1
    else:
        return 0

X["toss"] = train["toss"].apply(toss_attribute)
X["toss"].value_counts()

1    265
0    193
Name: toss, dtype: int64

In [38]:
X["opposition"] = train["opposition_rating"]
X["playing_role"] = le.fit_transform(train["playing_role"]) # One hot encoding is likely better. This is just illustration

In [39]:
X.head()

,consistency,form,opposition,venue,batting_hand,bowling_hand,match_time,home_venue,toss,playing_role
2991,3.5071,4.5163,85,5.0045,0,-2,0,1,1,8
2992,4.2889,3.5154,85,4.8535,1,1,0,1,1,7
2993,3.1134,3.9185,85,3.8896,1,-1,0,1,1,5
2994,4.7749,3.1801,85,4.9717,1,1,0,1,1,8
2995,3.5151,3.4464,85,4.3945,0,1,1,1,0,2


In [40]:
# One score before we go
model = LinearRegression()
score = cross_val_score(model, X.values, y.values, scoring="neg_mean_squared_error", n_jobs=-1)
print(f"Score: {score}\nMean: {score.mean()}")

Score: [-1007.10688971  -926.55911234  -730.87983388  -927.50300628
  -874.200867  ]
Mean: -893.2499418437426


#### Thanks for reading (or scrolling to the bottom :))